# analysis of keywords from cdc documents

In [1]:
import pandas as pd

raw = pd.read_csv('geo.coord_tweets/flair.joined.tweets.csv')

raw.head()

,created_at,statefips,countyfips,countyname,ogtext,cleantext,mentions,hashtags,pos,ner,sentiment
0,Mon Feb 10 07:03:44 +0000 2020,32.0,3.0,Clark,having a mocha and avoiding coronavirus at the...,have mocha avoid coronavirus airport,None,None,"[['having', 'VBG'], ['a', 'DT'], ['mocha', 'NN...",NaN,POSITIVE
1,Mon Feb 10 07:04:49 +0000 2020,36.0,81.0,Queens,our hearts beat with china,heart beat china,['linsapna'],None,"[['our', 'PRP$'], ['hearts', 'NNS'], ['beat', ...",NaN,POSITIVE
2,Mon Feb 10 07:05:30 +0000 2020,4.0,13.0,Maricopa,people stop my sister in the street to make fu...,people stop sister street fun wearing mask s a...,None,None,"[['people', 'NNS'], ['stop', 'VBP'], ['my', 'P...",NaN,POSITIVE
3,Mon Feb 10 07:10:48 +0000 2020,5.0,119.0,Pulaski,coronavirus death toll surpasses the number o...,coronavirus death toll surpass number fatal...,None,None,"[['coronavirus', 'NN'], ['death', 'NN'], ['tol...",NaN,NEGATIVE
4,Mon Feb 10 07:11:11 +0000 2020,5.0,119.0,Pulaski,coronavirus cases aboard japanese cruise ship...,coronavirus case aboard japanese cruise shi...,None,None,"[['coronavirus', 'NN'], ['cases', 'NNS'], ['ab...",NaN,NEGATIVE


In [17]:
raw.shape

(353588, 11)

# dataframe of all keywords

In [2]:
seedwords = [{"cat-id":i+1, "category":line.split(':')[0], "seed":seed} 
            for i,line in enumerate(open('seeds.txt','r')) 
            for seed in line.split(':')[1].strip().split(',')]

seeds = pd.DataFrame(seedwords)
seeds.head()

,cat-id,category,seed
0,1,Physical or social distancing,social distancing
1,1,Physical or social distancing,social distance
2,1,Physical or social distancing,socially distance
3,1,Physical or social distancing,physical distance
4,1,Physical or social distancing,physical distancing


# dataframe of if keyword exists in cleaned text

In [3]:
seedsdf = raw[['created_at', 'sentiment', 'statefips', 'countyfips', 'cleantext']]

seedlist = []

for seed in seeds['seed']:

    seedbools = [1 if str(seed) in str(text) else 0 for text in seedsdf['cleantext']]

    seedsdf[seed] = seedbools

    seedlist.append(seedbools)

seedsdf.head()

,created_at,sentiment,statefips,countyfips,cleantext,social distancing,social distance,socially distance,physical distance,physical distancing,...,antibody test,asymptomatic,diagnostic testing,fever,virtual,online,screen,disposable,barriers,exposure
0,Mon Feb 10 07:03:44 +0000 2020,POSITIVE,32.0,3.0,have mocha avoid coronavirus airport,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Mon Feb 10 07:04:49 +0000 2020,POSITIVE,36.0,81.0,heart beat china,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Mon Feb 10 07:05:30 +0000 2020,POSITIVE,4.0,13.0,people stop sister street fun wearing mask s a...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Mon Feb 10 07:10:48 +0000 2020,NEGATIVE,5.0,119.0,coronavirus death toll surpass number fatal...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Mon Feb 10 07:11:11 +0000 2020,NEGATIVE,5.0,119.0,coronavirus case aboard japanese cruise shi...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# keywords with highest occurance

In [21]:
seedsdf.iloc[:,5:].sum(axis=0).sort_values(ascending=False)[0:20]

test                 15228
mask                  9233
stay home             9159
social distancing     8812
ppe                   8405
quarantine            4705
wear mask             2888
fear                  2763
testing               2369
social distance       2231
worry                 2014
clean                 1562
online                1370
sanitize              1021
virtual                938
screen                 747
hand sanitizer         678
respirator             647
disinfect              642
isolation              613
dtype: int64

# keywords with lowest occurances

In [25]:
seedsdf.iloc[:,5:].sum(axis=0).sort_values(ascending=True)[0:30]

mental stress                        0
concentrates                         0
barriers                             0
routine cleaning                     0
regular cleaning and disinfecting    0
symptom screening                    0
skin protection                      0
wearing gloves                       0
N95 respirator                       0
Social support                       0
wear a mask                          0
socially connected                   0
school isolation                     0
stay-at-home                         0
6-feet                               0
prevention hygiene                   0
physical barriers                    0
duration of contact                  0
emotional well-being                 0
mental well-being                    0
6 feet apart                         0
continuity of health care            0
virtual meetings                     0
physically active                    1
personal protection equipment        1
sexual exploitation      

# tweets with top number of keywords included

In [23]:
seedsdf.iloc[:,5:].sum(axis=1).sort_values(ascending=False)[0:10]

68574     7
21040     7
91755     6
258807    6
73749     6
96618     6
10562     6
167584    6
74071     6
87695     6
dtype: int64

# number of tweets that have a keyword

In [27]:
tweet_occurance = seedsdf.iloc[:,5:].sum(axis=1).sort_values(ascending=False)
len(tweet_occurance[tweet_occurance > 0].index)

64394

# extra irrelevant code...

In [5]:
seedsdf.to_csv('found_seeds.tweets.csv', index=False)

In [1]:
# def check_cat(text, seedlist):
#     for seed in seedlist:
#         if str(seed) in text:
#             return 1
#     return 0

# 

# for i in range(7):
#     catdf[f'cat-i'] = [check_cat(str(text), seeds[seeds['cat-id'] == i+1])
#                         for text in catdf['cleantext']]

# catdf.head()

# catdf = raw[['created_at', 'sentiment', 'statefips', 'countyfips', 'cleantext']]

# for i in range(7):
#     catdf[f'cat-{i+1}'] = False

# for row in seedsdf.itertuples():
#     Ntotal = 0
#     for i in range(7):
#         cat = False
#         Ncat = len(seeds[seeds['cat-id'] == i+1])
#         for seed in row[6+Ntotal:6+Ntotal+Ncat]:
#             if seed == 1:
#                 cat = True
#         catdf.loc[row[0], f'cat-{i+1}'] = cat
        

In [24]:
Nseeds = [len(seeds[seeds['cat-id'] == i+1]) for i in range(7)]
Ntext = len(seedlist[0])

print(Nseeds)

for k in range(Ntext): # for each text
    N = 0
    for i, seedlen in enumerate(Nseeds):
        for seed in seedlist[N:N+seedlen]:
            print(seed)
            # if seed[k] == 1:
                
        N += seedlen




[19, 18, 2, 17, 16, 5, 24]
